# Title of the Analysis

**Authors:** 

## Overview

This is where an overview of the problem statement and a breif description of the analysis would go.

## Business Problem

Explain the business problem and include relevant sources to fully describe what the problem is, how it can be solved using models, and what is trying to be determined by the models. include background on the issue and related studies.

## Data Understanding

Describe where the data came from and how it relates to the business problem above. Give links to the data, a table with the relevant features and descriptions and links to other studies that give more information on the data set itself.

### Importing Packages and Loading the Datasets

In [38]:
#Import standard packages for data engineering and visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import date, datetime

#import packages to support geo visuals
import folium
from folium.plugins import HeatMap
import json

#Import packages to perform linear regression modeling and preprocessing tools
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, plot_roc_curve, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Import local functions from src/
import sys
sys.path.insert(0, 'src/')

#Ignore warnings in cells
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [39]:
# Import all related data sets

X = pd.read_csv('../data/water_well_training_values.csv')
y = pd.read_csv('../data/water_well_training_labels.csv')
df = pd.merge(y,X)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   status_group           59400 non-null  object 
 2   amount_tsh             59400 non-null  float64
 3   date_recorded          59400 non-null  object 
 4   funder                 55765 non-null  object 
 5   gps_height             59400 non-null  int64  
 6   installer              55745 non-null  object 
 7   longitude              59400 non-null  float64
 8   latitude               59400 non-null  float64
 9   wpt_name               59400 non-null  object 
 10  num_private            59400 non-null  int64  
 11  basin                  59400 non-null  object 
 12  subvillage             59029 non-null  object 
 13  region                 59400 non-null  object 
 14  region_code            59400 non-null  int64  
 15  di

### Get an understanding of the data

## Data Preparation

In [41]:
#check for duplicates in the data
df[df.duplicated()]

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group


We found no duplicate rows.

### Dealing with missing values 

In [42]:
# drop id column
df.drop('id',axis=1,inplace=True)

In [43]:
# Confirm the columns with missing values
df.isna().sum()

status_group                 0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [44]:
# Fill NaN values according to appropriate method
df['funder'].value_counts()

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
                          ... 
Mbeje                        1
Peter Mayiro                 1
Scholastica Pankrasi         1
Norad/rudep                  1
Samweli                      1
Name: funder, Length: 1897, dtype: int64

In [45]:
df['funder'].fillna(value='missing',inplace =True)

In [46]:
df['funder'].isna().sum()

0

In [47]:
df['installer'].value_counts()

DWE                17402
Government          1825
RWE                 1206
Commu               1060
DANIDA              1050
                   ...  
Yasini                 1
SELEPTA                1
Mama Kalage            1
Masele Nzengula        1
Amadi                  1
Name: installer, Length: 2145, dtype: int64

In [48]:
df['installer'].fillna(value='missing',inplace=True)

In [49]:
df['subvillage'].value_counts()

Madukani    508
Shuleni     506
Majengo     502
Kati        373
Mtakuja     262
           ... 
Lihiki A      1
Wisuzaje      1
Nyasosi       1
Rwamulya      1
Igowombe      1
Name: subvillage, Length: 19287, dtype: int64

In [50]:
df['subvillage'].fillna(value='none',inplace=True)

In [51]:
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [52]:
df['public_meeting'].fillna(value=False,inplace=True)

In [53]:
df['public_meeting'].value_counts()

True     51011
False     8389
Name: public_meeting, dtype: int64

In [54]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [55]:
df['scheme_management'].fillna(value='None',inplace=True)

In [56]:
df['scheme_name'].value_counts()

K                             682
None                          644
Borehole                      546
Chalinze wate                 405
M                             400
                             ... 
Mradi wa maji wa bombambil      1
TM part Three water supply      1
BL Kiungu Chelwa                1
Sanire water supply             1
Mradi wa mifugo                 1
Name: scheme_name, Length: 2696, dtype: int64

In [57]:
df.drop('scheme_name',axis=1,inplace=True)

In [58]:
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [59]:
df['permit'].fillna(value=False,inplace=True)

In [60]:
df.isna().sum()

status_group             0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
w

Explanation of why NaN values were filled using specific method

In [61]:
X_clean = df.drop('status_group',axis=1)

In [62]:
X_clean.isna().sum()

amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
w

Through description of the key features, data types, possible problems that need to be solved with cleaning and some suggestions of feature engineering.

In [90]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate


clfs = []
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier(n_jobs = 8))
clfs.append(GradientBoostingClassifier())

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())

NameError: name 'pipeline' is not defined